## Original textures 

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data
from torch.utils.data import DataLoader
import torch.nn as nn
import sys
import matplotlib.pyplot as plt
from tsne_torch import TorchTSNE
import numpy as np
sys.path.append('/home/gridsan/ckoevesdi/.local/lib/python3.9/site-packages/')
from pytorch_metric_learning import losses

sys.path.append('/home/gridsan/ckoevesdi/PooledStatisticsMetamers/poolstatmetamer/')
import utils.statnetencoder as sne
import importlib
import imp
import torch.nn.functional as F

#!wget https://raw.githubusercontent.com/pytorch/pytorch/master/torch/utils/collect_env.py

#sys.path.append(r'C:\Users\chris\Documents\MIT\Statistics_analysis_code\PyTorchSteerablePyramid')
sys.path.append('/home/gridsan/ckoevesdi/PyTorchSteerablePyramid/')
import steerable
import steerable.utils as utils
from steerable.SCFpyr_PyTorch import SCFpyr_PyTorch

torch.manual_seed(16)

#use GPU 2
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)
dtd_folder = 'home/gridsan/ckoevesdi/data/OT/dtd_torch/dtd/'


cuda:0


In [16]:
importlib.reload(sne)
#hyperparams
num_epochs = 100
batch_size = 1
crop_size = 128 
num_stats = 152 
optimizer_type='adam'
#optimizer_type='sgd'
learning_rate = 0.001
num_crops = 5 #changed this to four

multistat_penalty = 0
sparsity_penalty = 0 #0.01
entropic_penalty = 0

statnet_model = sne.StatNetEncoder(img_size=(crop_size,crop_size),
                                   batch_size=batch_size,
                                   num_stats=num_stats,
                                   device=device)
stat_labels = statnet_model.getsstatlabels(device)
ostat_labels = stat_labels

Creating solver: pyramid=UBBBBL_6 pooling=WholeImagePooling()


In [50]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class ImageDatasetFromTxt(Dataset):
    def __init__(self, txt_file, base_dir, transform=None):
        with open(txt_file, 'r') as f:
            self.image_paths = f.readlines()
        
        # Prepend base_dir to each path and remove newline characters
        self.image_paths = [base_dir + x.strip() for x in self.image_paths]
        
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")  # Convert the image to RGB format
        
        if self.transform:
            image = self.transform(image)
        
        return image

# Specify transformations (if needed)
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor()
])

txt_file = '/home/gridsan/ckoevesdi/data/OT/dtd_torch/dtd/dtd/labels/train10.txt'
base_dir = '/home/gridsan/ckoevesdi/DTD_col_cropped/'
dataset = ImageDatasetFromTxt(txt_file, base_dir, transform=transform)

batch_size = 1  # Set your desired batch size
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)



In [53]:
all_statisticsOT = []

for j, texture_batch in enumerate(dataloader):
    # Assuming texture_batch has a shape of [20, 5, 1, 128, 128]
    #print(texture_batch.shape)
    output = texture_batch.to(device)
    output = torch.stack([output, output]).squeeze(1)
    #print(output.shape)
    #print(labels)
    statistics = brucy(output)  # This should output a tensor of shape [2, 150]
    statistics = statistics[0,:]
    #print(statistics[0,:])
    # You can now store these statistics, along with the label and original image index
    all_statisticsOT.append({
        'statistics': statistics.cpu().numpy(),
        #'label': labels[i].item(),
        'original_image_index': labels.cpu().numpy()
    })

In [36]:
loading_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.Grayscale(),
    transforms.ToTensor(),  # This will also convert the image from [0, 255] to [0.0, 1.0]
    #transforms.ConvertImageDtype(torch.float32)
])

#use training set for now
class_dtd_datasetOT = torchvision.datasets.DTD(root='/home/gridsan/ckoevesdi/data/OT/dtd_torch/', split='train', 
                                             partition=10, 
                                       transform=loading_transforms, target_transform=None,
                                       download=False) #ah das datenset muss so aussehen wie es auf der website auch ist, deswegen kann man auch download false machen
# Define the batch size (Change this based on your requirements)
batch_size = 1

# Create a DataLoader
classification_dataloaderOT = DataLoader(
    class_dtd_datasetOT,
    batch_size=batch_size,
    shuffle=False
)
#tensor2pil_transform = transforms.ToPILImage()

In [37]:
import utils.brucenet as bn
brucy = bn.BruceNet(pooling_region_size=1e20, pyramid_params=False, dummy_img = torch.zeros(20,
                                     1,
                                     256,
                                     256)).to(device)

Creating solver: pyramid=UBBBBL_6 pooling=WholeImagePooling()


In [38]:
all_statisticsOT = []

for j, (texture_batch, labels) in enumerate(classification_dataloaderOT):
    # Assuming texture_batch has a shape of [20, 5, 1, 128, 128]
    #print(texture_batch.shape)
    output = texture_batch.to(device)
    output = torch.stack([output, output]).squeeze(1)
    #print(output.shape)
    #print(labels)
    statistics = brucy(output)  # This should output a tensor of shape [2, 150]
    statistics = statistics[0,:]
    #print(statistics[0,:])
    # You can now store these statistics, along with the label and original image index
    all_statisticsOT.append({
        'statistics': statistics.cpu().numpy(),
        #'label': labels[i].item(),
        'original_image_index': labels.cpu().numpy()
    })

In [54]:
statistics_arrayOT = np.array([item['statistics'] for item in all_statisticsOT])
print(statistics_arrayOT.shape)

(4183, 152)


## Heeger and Bergen

In [40]:
dtd_folder = '/home/gridsan/ckoevesdi/data/dtd_torch/dtd/dtd/'

In [61]:
loading_transforms = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor(),  # This will also convert the image from [0, 255] to [0.0, 1.0]
])

#use training set for now
class_dtd_datasetHB = torchvision.datasets.DTD(root='/home/gridsan/ckoevesdi/data/dtd_torch/', split='train', 
                                             partition=10, 
                                       transform=loading_transforms, target_transform=None,
                                       download=False) #ah das datenset muss so aussehen wie es auf der website auch ist, deswegen kann man auch download false machen
# Define the batch size (Change this based on your requirements)
batch_size = 1

# Create a DataLoader
classification_dataloaderHB = DataLoader(
    class_dtd_datasetHB,
    batch_size=batch_size,
    shuffle=False
)


In [62]:
all_statisticsHB = []

for j, (texture_batch, labels) in enumerate(classification_dataloaderHB):
    # Assuming texture_batch has a shape of [20, 5, 1, 128, 128]
    #print(texture_batch.shape)
    output = texture_batch.to(device)
    output = torch.stack([output, output]).squeeze(1)
    #print(output.shape)
    #print(labels)
    statisticsHB = brucy(output)  # This should output a tensor of shape [2, 150]
    statisticsHB = statisticsHB[0,:]
    #print(statistics[0,:])
    # You can now store these statistics, along with the label and original image index
    all_statisticsHB.append({
        'statistics': statistics.cpu().numpy(),
        #'label': labels[i].item(),
        'original_image_index': labels.cpu().numpy()
    })

In [63]:
statistics_arrayHB = np.array([item['statistics'] for item in all_statisticsHB])
print(statistics_arrayHB.shape)

(4183, 152)


In [64]:
relative_errors = np.mean(np.abs(statistics_arrayOT - statistics_arrayHB) / (np.abs(statistics_arrayOT) + 1e-10), axis=0)

In [65]:
# Get the indices that would sort the array
sorted_indices = np.argsort(relative_errors)
# Assuming you have arrays relative_errors and sorted_indices
np.save('relative_errors.npy', relative_errors)
np.save('sorted_indices.npy', sorted_indices)
# Print the ranked feature indices
print("Ranked feature indices from best to worst:")
print(sorted_indices)

Ranked feature indices from best to worst:
[ 64  72  42  54  46  76  50  68  86  92  20  28  24  32  95  89  40  62
  59  57  49  38  36  35  27  41  39  60  33  25  58  81 106 104  34  37
 109  63 107  26  21  61  29  22 120  30  43  55 118  84  88 123   4  51
  47 121  85  82  52  71  80  83  79  31  44  56   0  23  73  48 137  65
 135  45 132  53 134  77  99  74  69 102  66 101  78 103  93  70  98 100
 149 151  87 146 148  75  90  96 128  67 147  91  97 142   1   5   8  11
  14  94  17 143 141 131 127 115 124 113 125 117 138 111 129 139 133   2
 145   6   9 130  12 144 136 108  15 122  18   3   7  10 119 150  13 116
  16  19 126 140 110 112 105 114]


In [66]:
print("Ranked features from best to worst:")
i = 1
for idx in sorted_indices:
    i = i + 1
    print(f"Rank {i}: {ostat_labels[idx]}, Relative Error: {relative_errors[idx]:.4f}")

Ranked features from best to worst:
Rank 2:  edge_mean L3 Ori0, Relative Error: 0.4114
Rank 3:  edge_mean L3 Ori2, Relative Error: 0.4330
Rank 4:  edge_mean L2 Ori0, Relative Error: 0.4447
Rank 5:  edge_mean L2 Ori3, Relative Error: 0.4565
Rank 6:  edge_mean L2 Ori1, Relative Error: 0.4630
Rank 7:  edge_mean L3 Ori3, Relative Error: 0.4655
Rank 8:  edge_mean L2 Ori2, Relative Error: 0.4663
Rank 9:  edge_mean L3 Ori1, Relative Error: 0.4857
Rank 10:  edge_mean L4 Ori0, Relative Error: 0.4884
Rank 11:  edge_mean L4 Ori2, Relative Error: 0.4993
Rank 12:  edge_mean L1 Ori0, Relative Error: 0.5319
Rank 13:  edge_mean L1 Ori2, Relative Error: 0.5388
Rank 14:  edge_mean L1 Ori1, Relative Error: 0.5428
Rank 15:  edge_mean L1 Ori3, Relative Error: 0.5468
Rank 16:  edge_mean L4 Ori3, Relative Error: 0.6620
Rank 17:  edge_mean L4 Ori1, Relative Error: 0.6640
Rank 18:  edge_correlation L1 Ori1_3, Relative Error: 0.7276
Rank 19:  edge_correlation L2 Ori1_3, Relative Error: 0.7289
Rank 20:  edge_cor

In [20]:
print(1e-10+1)

1.0000000001
